### Purpose:

1) Merge raw data files into single feature table

2) split table into multiple sets to resolve memory overload problems

In [113]:
import pandas as pd

# load raw data files
df_train = pd.read_csv('../../data/raw/sales_train.csv')
df_items = pd.read_csv('../../data/raw/items_weekly_sell_prices.csv')
df_cal = pd.read_csv('../../data/raw/calendar.csv')
df_events = pd.read_csv('../../data/raw/calendar_events.csv')

In [114]:
stores = df_train['store_id'].unique()

In [115]:
## save subset files for each store
for store in stores:
    df = df_train.loc[df_train['store_id'] == store]
    df.to_csv(f'../../data/interim/sales_train_{store}.csv', index=False)

In [116]:
## clear memory of df_train
del(df_train)

In [1]:
## develop functions to clean df_train
def convert_train(df):
    df.pop('id')
    df = pd.melt(df, id_vars=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'd', value_name = 'items_sold')

    return df

def merge_cal(df):
    df_cal = pd.read_csv('../../data/raw/calendar.csv')
    df = pd.merge(df, df_cal, on='d', how='left')
    df.pop('d')

    del(df_cal)
    return df

def merge_item_prices(df):
    df_items = pd.read_csv('../../data/raw/items_weekly_sell_prices.csv')
    df = pd.merge(df, df_items, left_on = ['store_id', 'item_id', 'wm_yr_wk'], right_on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
    df.pop('wm_yr_wk')
    df['items_sold'] = df['items_sold'].fillna(0)
    df['sell_price'] = df['sell_price'].fillna(0)

    df['revenue'] = df['items_sold'] * df['sell_price']
    df.pop('items_sold')
    df.pop('sell_price')

    del(df_items)
    return df

def merge_events(df):
    df_events = pd.read_csv('../../data/raw/calendar_events.csv')
    df_events = pd.pivot_table(df_events, index='date', values='event_name', columns='event_type', aggfunc='count', fill_value= 0)
    df_events = df_events.rename_axis(columns= None).reset_index()
    ### rename cols for better readbility
    df_events = df_events.rename(columns={"Cultural": "event_cultural", "National": "event_national", "Religious": "event_religious", "Sporting": "event_sport"})

    df = pd.merge(df, df_events, left_on = ['date'], right_on = ['date'], how = 'left')
    df = df.fillna(0)
    del(df_events)
    return df

def separate_dates(df):
    df['date'] = pd.to_datetime(df['date'])
    df['day_of_week'] = df['date'].dt.day_of_week
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df.pop('date')
    return df

In [246]:
df_train = pd.read_csv('../../data/interim/sales_train_CA_1.csv')

In [219]:
df_train

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1532,d_1533,d_1534,d_1535,d_1536,d_1537,d_1538,d_1539,d_1540,d_1541
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,1,0,1,0,1,0,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8,2,0,8,2,3,1,1,3,8
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,0,1,3,2,1,1,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,FOODS_3_823_CA_1_evaluation,FOODS_3_823,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,1,0,0,2,0,0,1,0,0,3
3045,FOODS_3_824_CA_1_evaluation,FOODS_3_824,FOODS_3,FOODS,CA_1,CA,1,0,5,0,...,0,0,0,0,0,0,0,0,0,0
3046,FOODS_3_825_CA_1_evaluation,FOODS_3_825,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,0,4,1,0,1,1,1,1,0,2
3047,FOODS_3_826_CA_1_evaluation,FOODS_3_826,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,3,2,0,2,0,0,1,4,2,0


In [247]:
df_train = convert_train(df_train)
df_train = merge_cal(df_train)
df_train = merge_item_prices(df_train)
df_train = merge_events(df_train)
df_train = separate_dates(df_train)

In [248]:
df_train

,item_id,dept_id,cat_id,store_id,state_id,revenue,event_cultural,event_national,event_religious,event_sport,day_of_week,month,year
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.00,0.0,0.0,0.0,0.0,5,1,2011
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.00,0.0,0.0,0.0,0.0,5,1,2011
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.00,0.0,0.0,0.0,0.0,5,1,2011
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0.00,0.0,0.0,0.0,0.0,5,1,2011
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0.00,0.0,0.0,0.0,0.0,5,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4698504,FOODS_3_823,FOODS_3,FOODS,CA_1,CA,8.64,0.0,0.0,0.0,0.0,5,4,2015
4698505,FOODS_3_824,FOODS_3,FOODS,CA_1,CA,0.00,0.0,0.0,0.0,0.0,5,4,2015
4698506,FOODS_3_825,FOODS_3,FOODS,CA_1,CA,7.96,0.0,0.0,0.0,0.0,5,4,2015
4698507,FOODS_3_826,FOODS_3,FOODS,CA_1,CA,0.00,0.0,0.0,0.0,0.0,5,4,2015


In [222]:
df_train.memory_usage(deep=True)

Index                        132
item_id                327693650
dept_id                308899614
cat_id                 299502596
store_id               286609049
state_id               277212031
revenue                 37588072
Chanukah End            37588072
Christmas               37588072
Cinco De Mayo           37588072
ColumbusDay             37588072
Easter                  37588072
Eid al-Fitr             37588072
EidAlAdha               37588072
Father's day            37588072
Halloween               37588072
IndependenceDay         37588072
LaborDay                37588072
LentStart               37588072
LentWeek2               37588072
MartinLutherKingDay     37588072
MemorialDay             37588072
Mother's day            37588072
NBAFinalsEnd            37588072
NBAFinalsStart          37588072
NewYear                 37588072
OrthodoxChristmas       37588072
OrthodoxEaster          37588072
Pesach End              37588072
PresidentsDay           37588072
Purim End 

In [223]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4698509 entries, 0 to 4698508
Data columns (total 43 columns):
 #   Column               Dtype  
---  ------               -----  
 0   item_id              object 
 1   dept_id              object 
 2   cat_id               object 
 3   store_id             object 
 4   state_id             object 
 5   revenue              float64
 6   Chanukah End         float64
 7   Christmas            float64
 8   Cinco De Mayo        float64
 9   ColumbusDay          float64
 10  Easter               float64
 11  Eid al-Fitr          float64
 12  EidAlAdha            float64
 13  Father's day         float64
 14  Halloween            float64
 15  IndependenceDay      float64
 16  LaborDay             float64
 17  LentStart            float64
 18  LentWeek2            float64
 19  MartinLutherKingDay  float64
 20  MemorialDay          float64
 21  Mother's day         float64
 22  NBAFinalsEnd         float64
 23  NBAFinalsStart       float64
 24

In [7]:
## reduce memory utilisation
def convert_types(df):
    #df['cat_id'] = df['cat_id'].astype("category")
    #df['dept_id'] = df['dept_id'].astype("category")
    #df['store_id'] = df['store_id'].astype("category")
    #df['state_id'] = df['state_id'].astype("category")
    #df['item_id'] = df['item_id'].astype("category")
    #df['event_name'] = df['event_name'].astype('category')
    #df['event_type'] = df['event_type'].astype('category')
    #df['revenue'] = pd.to_numeric(df['revenue'], downcast='float')
    #df['day_of_week'] = pd.to_numeric(df['day_of_week'], downcast='unsigned')
    #df['year'] = pd.to_numeric(df['year'], downcast='unsigned')
    #df['month'] = pd.to_numeric(df['month'], downcast='unsigned')

    cat_cols = df.select_dtypes('object').columns
    df[cat_cols] = df[cat_cols].astype('category')
    
    df['revenue'] = pd.to_numeric(df['revenue'], downcast='float')

    num_cols = df.select_dtypes('float').columns
    if 'revenue' in num_cols:
        num_cols.drop('revenue')
    
    df[num_cols] = df[num_cols].astype(int)

    num_cols = df.select_dtypes('integer').columns
    df[num_cols] = df[num_cols].apply(pd.to_numeric, downcast="unsigned")
    

    return df
    

In [250]:
df_train = convert_types(df_train)

In [251]:
df_train.memory_usage(deep=True)

Index                  132
item_id            9675756
dept_id            4699269
cat_id             4698809
store_id           4698678
state_id           4698676
revenue            9397018
event_cultural     4698509
event_national     4698509
event_religious    4698509
event_sport        4698509
day_of_week        4698509
month              4698509
year               9397018
dtype: int64

In [252]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4698509 entries, 0 to 4698508
Data columns (total 13 columns):
 #   Column           Dtype   
---  ------           -----   
 0   item_id          category
 1   dept_id          category
 2   cat_id           category
 3   store_id         category
 4   state_id         category
 5   revenue          uint16  
 6   event_cultural   uint8   
 7   event_national   uint8   
 8   event_religious  uint8   
 9   event_sport      uint8   
 10  day_of_week      uint8   
 11  month            uint8   
 12  year             uint16  
dtypes: category(5), uint16(2), uint8(6)
memory usage: 71.8 MB


In [3]:
def pre_engineering(df):
    df = convert_train(df)
    df = merge_cal(df)
    df = merge_item_prices(df)
    df = merge_events(df)
    df = separate_dates(df)
    df = convert_types(df)
    
    return df


In [255]:
df_train = pd.DataFrame()

In [9]:
import pandas as pd
from glob import glob
df_train = pd.DataFrame()

for f in glob('../../data/interim/sales_train_*.csv'):
    df = pd.read_csv(f)
    df = pre_engineering(df)

    if df_train.empty:
        df_train = df
    else:
        df_train = pd.concat([df_train, df], ignore_index=True)
    
    del(df)
del(glob)       ## recover memory
        

In [ ]:
df_train.info()

In [11]:
cat_cols = df_train.select_dtypes('object').columns
df_train[cat_cols] = df_train[cat_cols].astype('category')

In [22]:
df_train.to_parquet('../../data/processed/df_clean.parquet')